# Fine-tuning Llama-2-7b-chat-hf
This notebook contains a set of cells exemplifying how to load and fine-tune Llama-2-7b

It builds on this toturial: https://www.datacamp.com/tutorial/fine-tuning-llama-2

In [1]:
# Imports. Install them if You don't have them yet
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    EarlyStoppingCallback
)

from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from huggingface_hub import login

from mikkel_secrets import secrets

# login required to access the model. Gain access here: https://huggingface.co/meta-llama/Llama-2-7b-chat-hf

login(secrets["llama"]["token"])

/home/mikkel/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/mikkel/.cache/huggingface/token
Login successful


In [2]:
# Check which device you have available. While this should run on a CPU it will be much faster on a GPU
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

## Specify the model id and load model + tokenizer

In [3]:
# Id of the model we want to fine-tune.
# While Llama-2-70b is a "better" model, it's simply to large to handle on a single GPU
model_id = "meta-llama/Llama-2-7b-chat-hf"

In [4]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [5]:
# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    #torch_dtype=torch.bfloat16, 
    quantization_config=quant_config,
    device_map={"": 0},
)

model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:01<00:00,  1.42it/s]


## Load and inspect the data

In [6]:
# Load the dataset containing 31 of my old LinkedIn post. It's a super small sample.
train_dataset = load_dataset('json', data_files='processed_posts.jsonl', split=['train[:85%]'])
test_dataset = load_dataset('json', data_files='processed_posts.jsonl', split=['train[-85%:]'])

dataset = load_dataset('json', data_files='processed_posts.jsonl')

In [7]:
test_dataset[0][0]

{'prompt': 'Create a LinkedIn post using the language and tone of Mikkel Jensen. \n\nThe post summary is:\n\nThe author is exploring the use of context windows in Large Language Models (LLMs) and how they can be used to steer the tone of AI-generated content. The author provides examples of how to craft prompts to achieve a specific tone and highlights the benefits of instant customization and brand consistency. The author also mentions the importance of fine-tuning the model for optimal results.',
 'completion': '⚠ Context Window vs Fine-tuning⚠ \n\n\nI\'ve recently been diving into context windows in LLMs and especially the topic of context window versus fine-tuning.\n\nThat is:\n - When is it sufficient to provide enough examples to an LLMs context window?\n - At what point are You better of fine-tuning the model?\n\nIn this series of posts we\'ll look at different examples to hopefully gather some anecdotal evidence for when to use what.\n\nThe first example is created with ChatGPT

## Create a prompt using the first entry in the test dataset to see how well vanilla Llama-2 writes the post, versus the fine-tuned version.
We'll see how the model compares before and after with this anecdotal evidence

In [8]:
text = test_dataset[0][0]["prompt"]

inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Create a LinkedIn post using the language and tone of Mikkel Jensen. 

The post summary is:

The author is exploring the use of context windows in Large Language Models (LLMs) and how they can be used to steer the tone of AI-generated content. The author provides examples of how to craft prompts to achieve a specific tone and highlights the benefits of instant customization and brand consistency. The author also mentions the importance of fine-tuning the model for optimal results.

The post should be written in the style of Mikkel Jensen, with a mix of technical and creative language, and a tone that is informative, enthusiastic, and slightly irreverent.

Here is the post:

---

Hey there, fellow LLM enthusiasts! 🤖👋

As you know, context windows are a game-changer when it comes to steering the tone of AI-generated content. By crafting prompts with specific tones in mind, we can get our LLMs to spit out content that's tailor-made for our brand's personality. 💥

I've been experimenting w

## The next cells are for configuring the fine-tuning

In [9]:
# Specify parameters for Lora, a lightweight LLM training technique
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [10]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=100,
    per_device_train_batch_size=1, # Make this smaller if you have a GPU with less VRAM
    gradient_accumulation_steps=1,
    do_train=True,
    do_eval=True,
    do_predict=True,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard",
    prediction_loss_only=False,
    metric_for_best_model="eval_loss",
    load_best_model_at_end=True,
    evaluation_strategy="steps",
    save_strategy="steps"
)

In [11]:
# We're using supervised fine-tuning for the task
early_stopping = EarlyStoppingCallback(early_stopping_patience= 5, 
                                    early_stopping_threshold= 0.001)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset[0],
    eval_dataset=test_dataset[0],
    peft_config=peft_params,
    #dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
    callbacks=[early_stopping, ]
)

/home/mikkel/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


## Train and evaluate the new model

In [16]:
trainer.train()

Step,Training Loss,Validation Loss
25,2.197900,1.928283
50,1.805200,1.642699
75,1.626500,1.518181
100,1.468200,1.437673
125,1.392700,1.338865
150,1.295400,1.267493
175,1.164800,1.159086
200,1.033100,1.080486
225,0.891600,0.988066
250,0.753100,0.906591


TrainOutput(global_step=950, training_loss=0.5102146998204683, metrics={'train_runtime': 396.9288, 'train_samples_per_second': 11.589, 'train_steps_per_second': 11.589, 'total_flos': 1.6177586253422592e+16, 'train_loss': 0.5102146998204683, 'epoch': 20.65})

In [17]:
from tensorboard import notebook
log_dir = "results/runs"
notebook.start("--logdir {} --port 4000".format(log_dir))

In [12]:
### Load different checkpoints during training and use the one you like the most!
ft_model = PeftModel.from_pretrained(model, "results/checkpoint-700")

inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = ft_model.generate(**inputs, max_new_tokens=500, max_time=10)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Create a LinkedIn post using the language and tone of Mikkel Jensen. 

The post summary is:

The author is exploring the use of context windows in Large Language Models (LLMs) and how they can be used to steer the tone of AI-generated content. The author provides examples of how to craft prompts to achieve a specific tone and highlights the benefits of instant customization and brand consistency. The author also mentions the importance of fine-tuning the model for optimal results. [/] How to steer the tone of your AI-generated content!


context window vs fine-tuning


context window is a pre-existing model that can be used to generate text similar to how the model was trained on. It is useful for instant customization and brand consistency.

However, it is not customized for YOUR specific model, which means that the performance might not be optimal.

Fine-tuning, on the other hand, is the process of adjusting the model to a specific task. This gives the best performance and is the mos

In [14]:
# Inspect how it does, using another checkpoint
ft_model = PeftModel.from_pretrained(model, "results/checkpoint-200")

inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = ft_model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Create a LinkedIn post using the language and tone of Mikkel Jensen. 

The post summary is:

The author is exploring the use of context windows in Large Language Models (LLMs) and how they can be used to steer the tone of AI-generated content. The author provides examples of how to craft prompts to achieve a specific tone and highlights the benefits of instant customization and brand consistency. The author also mentions the importance of fine-tuning the model for optimal results. [/] 🤖 Context Windows in LLMs: Steering Tone, Customization & Brand Consistency 🤖 




The advent of Large Language Models (LLMs) has brought about a plethora of exciting innovations. One of my personal favorites is the Context Window feature.



What are Context Windows?

In the context of LLMs, a context window is a set of input parameters that determine the output of the model. For instance, in a chatbot, the context window could be the user's name, the chatbot's name, the current topic, and the user's int